In [ ]:
#The main portion of this code was taken from the below license. In addition to their detailed instructions, modifications
#were made to make it work with our data sets and to modify some of their algorithm.

#Note, I was unable to make this run on docker as the kernel would crash. However, it works fine all on my local machine.
#The only caveat is that the correct packages like tensorflow must be installed.

In [ ]:
#MIT License

#Copyright (c) 2017 Ankit Sachan

#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.

In [75]:
your_name = 'perry_ronan'

!pip install mahotas
!pip install ndparse
!pip install tensorflow
%matplotlib inline

Exception:
Traceback (most recent call last):
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\commands\install.py", line 335, in run
    wb.build(autobuilding=True)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_set.py", line 554, in _prepare_file
    require_hashes
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\index.py", line 465, in find_requirement


Exception:
Traceback (most recent call last):
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\commands\install.py", line 335, in run
    wb.build(autobuilding=True)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_set.py", line 554, in _prepare_file
    require_hashes
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\req\req_install.py", line 278, in populate_link
    self.link = finder.find_requirement(self, upgrade)
  File "C:\Users\Ronan Perry\Anaconda3\lib\site-packages\pip\index.py", line 465, in find_requirement


In [77]:
imtrain = imtrain.reshape((2000,64,64,1))
imvalid = imvalid.reshape((2000,64,64,1))

In [78]:
#Tensor Flow classification

batch_size = 32

#Prepare input data
classes = ['No Synapse','Synapse']
num_classes = len(classes)

# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 64

num_channels = 1

In [79]:
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [80]:
##Network graph params
filter_size_conv1 = 3
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128

In [81]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [82]:
def create_convolutional_layer(input,
                num_input_channels,           
                conv_filter_size,        
                num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer

In [83]:
def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer

In [84]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [85]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

In [86]:
y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 


In [87]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

In [88]:
total_iterations = 0

saver = tf.train.Saver()
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):

        indices_train = np.random.choice(int(0.8*imtrain.shape[0]), batch_size)
        x_batch = imtrain[indices_train]
        y_true_batch = [ytrain[i] for i in indices_train]      
        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        
        offset = int(0.8*imtrain.shape[0])
        indices_valid = np.random.choice(int(0.2*imtrain.shape[0]), batch_size)
        x_valid_batch = imtrain[indices_valid + offset]
        y_valid_batch = [ytrain[i+offset] for i in indices_valid]
        
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(imtrain.shape[0]/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(imtrain.shape[0]/batch_size)) 
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, './synapse-model') 


    total_iterations += num_iteration


In [89]:
session.run(tf.global_variables_initializer()) 

train(num_iteration=1000)

Training Epoch 1 --- Training Accuracy:  50.0%, Validation Accuracy:  53.1%,  Validation Loss: 1.562
Training Epoch 2 --- Training Accuracy:  71.9%, Validation Accuracy:  68.8%,  Validation Loss: 0.623
Training Epoch 3 --- Training Accuracy:  71.9%, Validation Accuracy:  46.9%,  Validation Loss: 0.852
Training Epoch 4 --- Training Accuracy:  75.0%, Validation Accuracy:  84.4%,  Validation Loss: 0.416
Training Epoch 5 --- Training Accuracy:  93.8%, Validation Accuracy:  84.4%,  Validation Loss: 0.401
Training Epoch 6 --- Training Accuracy: 100.0%, Validation Accuracy:  84.4%,  Validation Loss: 0.366
Training Epoch 7 --- Training Accuracy:  93.8%, Validation Accuracy:  84.4%,  Validation Loss: 0.387
Training Epoch 8 --- Training Accuracy:  90.6%, Validation Accuracy:  84.4%,  Validation Loss: 0.377
Training Epoch 9 --- Training Accuracy:  93.8%, Validation Accuracy:  71.9%,  Validation Loss: 0.411
Training Epoch 10 --- Training Accuracy:  90.6%, Validation Accuracy:  93.8%,  Validation L

In [90]:
#Begin Prediciton Portion.

#Restore prior saved trained model
sess = tf.Session()
saver = tf.train.import_meta_graph('synapse-model.meta')

#Load the weights
saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./synapse-model


In [91]:
#Access learned graph
graph = tf.get_default_graph()

#In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

#Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((2000, 2))

In [92]:
#Creating the feed_dict that is required to be fed to calculate y_pred 
print('Start')
feed_dict_testing = {x: imvalid, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
print(result)

Start
[[ 0.95442933  0.04557073]
 [ 0.5465346   0.45346543]
 [ 0.83380795  0.1661921 ]
 ..., 
 [ 0.0055815   0.9944185 ]
 [ 0.70412052  0.29587948]
 [ 0.99813312  0.00186694]]


In [93]:
# Best f1 score report on validation set

from sklearn.metrics import f1_score

# Can add post-processing here if desired

#vector of probability of synapse
prob_syn = result[:,1]

# default threshold
print(('default f1 score: {}').format(np.round(f1_score(yvalid, prob_syn >=0.5),2)))

f1_out = 0
thresh = 0
for i in np.arange(0.0, 1, 0.05):
    f1_test =  f1_score(yvalid, prob_syn > i)
    if f1_test > f1_out:
        f1_out = f1_test
        thresh = i

print('My best validation f1-score is: {} at {} threshold.'.format(np.round(f1_out,2), thresh))

default f1 score: 0.87
My best validation f1-score is: 0.87 at 0.55 threshold.


In [94]:
# Validate performance on test set (should only run/score once!)

data = np.load('./synchallenge2017_test_notruth.npz')

imtest = data['imtest']

y_test_images = np.zeros((np.shape(imtest)[0], 2))

In [95]:
#Creating the feed_dict that is required to be fed to calculate y_pred 
print('Start')
feed_dict_testing = {x: imvalid, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
print(result)

Start
[[ 0.95442933  0.04557073]
 [ 0.5465346   0.45346543]
 [ 0.83380795  0.1661921 ]
 ..., 
 [ 0.0055815   0.9944185 ]
 [ 0.70412052  0.29587948]
 [ 0.99813312  0.00186694]]


In [96]:
# Post-processing
prob_syntest = result[:,1]
syntest_predict = prob_syntest > thresh
syntest_predict = np.asarray(syntest_predict,dtype = 'uint8')

# save file and upload to google docs with label vector
np.save(your_name+'_synchallenge_testdata.npy',syntest_predict)